In [1]:
from pyshipt_streams import KafkaProducer, KafkaConsumer
from uuid import uuid4
import os

staging_brokers = "pkc-438qm.us-east-1.aws.confluent.cloud:9092"
kafka_username = os.environ["KAFKA_KEY"],
kafka_password = os.environ["KAFKA_SECRET"]

In [2]:
p = KafkaProducer(
    brokers=staging_brokers,
    username=kafka_username,
    password=kafka_password
)

{"level": "INFO", "message": "Connected to Kafka Broker", "timestamp": "2021-10-12T21:52:10.476910Z", "app_env": null, "host": null, "version": null}


In [ ]:
from confluent_kafka import Producer
p = Producer(
    {
        "bootstrap.servers": "pkc-438qm.us-east-1.aws.confluent.cloud:9092",
        "sasl.username": kafka_username,
        "sasl.password": kafka_password,
        "security.protocol": "SASL_SSL",
        "sasl.mechanism": "PLAIN",
    }
)

# Publish a message to staging input

In [8]:
for x in range(10):
    p.publish(
        "stg.bus.ds-marketplace.v1.bundle_engine_input",
        {
            "event_id": str(uuid4()),
            "message": "hello-world",
        }
    )

{"level": "INFO", "message": "Sent to topic: stg.bus.ds-marketplace.v1.bundle_engine_input", "timestamp": "2021-10-12T21:54:58.315822Z", "app_env": null, "host": null, "version": null}
{"level": "INFO", "message": "Message produced for topic: stg.bus.ds-marketplace.v1.bundle_engine_input", "timestamp": "2021-10-12T21:54:58.318537Z", "app_env": null, "host": null, "version": null}
{"level": "INFO", "message": "Sent to topic: stg.bus.ds-marketplace.v1.bundle_engine_input", "timestamp": "2021-10-12T21:54:58.319328Z", "app_env": null, "host": null, "version": null}
{"level": "INFO", "message": "Sent to topic: stg.bus.ds-marketplace.v1.bundle_engine_input", "timestamp": "2021-10-12T21:54:58.320049Z", "app_env": null, "host": null, "version": null}
{"level": "INFO", "message": "Sent to topic: stg.bus.ds-marketplace.v1.bundle_engine_input", "timestamp": "2021-10-12T21:54:58.320987Z", "app_env": null, "host": null, "version": null}
{"level": "INFO", "message": "Sent to topic: stg.bus.ds-market

# Listen for message on the output topic

In [4]:
c = KafkaConsumer(
    consumer_group="my_username",
    brokers=staging_brokers,
    username=kafka_username,
    password=kafka_password,
)
c.subscribe(topics=["stg.bus.ds-marketplace.v1.bundle_engine_output"])
messages = []

{"level": "INFO", "message": "Connected to Kafka Broker", "timestamp": "2021-10-12T21:52:34.501883Z", "app_env": null, "host": null, "version": null}


In [ ]:
try:
    while True:
        msg = c.poll(0.25)
        if msg is None:
            continue
        if msg.error():
            raise Exception("errr")
        else:
            # Do something with the message
            m = msg.value().decode("utf-8")
            messages.append(m)
            print(m)
            
except KeyboardInterrupt:
    print("aborted")
finally:
    c.close()